<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab6/lab8_Sarcasm_Detection-assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2
!git pull
%cd lab6

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ai-intensive2
Already up to date.
/content/drive/MyDrive/ai-intensive2/lab6


# Lab 8 : Sarcasm Detection

@copyright:
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.

---


## 목차

1. Sarcasm Detection
2. Data Preprocessing
3. Modeling
4. Training / Evaluation

## 1. Sarcasm Detection

`Sarcasm Detection` task는 target 문장이 sarcastic한지 아닌지를 판별하는 task이다.

[ACL2020 dataset](https://github.com/EducationalTestingService/sarcasm)을 사용한다. 해당 dataset은 3개의 column으로 구성되어 있다.

- `label`: sarcastic 여부를 나타내는 binary value (NOT-SARCASM: non-sarcastic, SARCASM: sarcastic)
- `response`: target 문장
- `context`: target 문장 앞 문장들 (최대 20)

### 본 실습에서는, 지금까지 배운 내용을 활용하여 Dataset의 이해 및 전처리, 배운 Model의 Implement 및 비교, 그리고 학습 및 평가를 진행한다.

- 원하는 전처리, 모델, 학습 방법을 적절히 선택하여 진행하고, 여러 실험을 통해 최적의 결과를 얻도록 한다.
- 그 과정에서 변경하는 모델의 구조 및 전처리 방법, 학습 전략, 하이퍼파라미터 등을 기록하고, 그에 따른 결과 및 해석을 함께 기록한다. (해당 기록을 평가)
- 최종적으로, 75% 이상의 accuracy를 달성하도록 한다.
- 본 실습에서는 가장 간단한 BertForSequenceClassification 모델을 사용한다. 해당 코드를 참고하여 진행하면 된다.

In [24]:
!git clone https://github.com/EducationalTestingService/sarcasm.git

fatal: destination path 'sarcasm' already exists and is not an empty directory.


## 2. Data Preprocessing

In [25]:
import json
import pprint

with open("./sarcasm/twitter/sarcasm_detection_shared_task_twitter_training.jsonl", 'r', encoding="utf-8") as file:
    datalines = file.readlines()

data = []
for line in datalines:
    data.append(json.loads(line))

print("Data 개수: ", len(data))
print("Data 예시: ")
pprint.pprint(data[0])

Data 개수:  5000
Data 예시: 
{'context': ['A minor child deserves privacy and should be kept out of '
             'politics . Pamela Karlan , you should be ashamed of your very '
             'angry and obviously biased public pandering , and using a child '
             'to do it .',
             "@USER If your child isn't named Barron ... #BeBest Melania "
             "couldn't care less . Fact . 💯"],
 'label': 'SARCASM',
 'response': "@USER @USER @USER I don't get this .. obviously you do care or "
             "you would've moved right along .. instead you decided to care "
             'and troll her ..'}


In [26]:
!pip install emoji

In [27]:
# Data 전처리. Data를 직접 살펴보며 Emoji, URL, Special Character 등 적절한 처리를 진행하도록 한다.
import json
import re
import emoji

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'@user', '', sentence)
    sentence = re.sub(r'<URL>', '', sentence)
    sentence = re.sub(r'http[s]?://\S+', '', sentence)

    return sentence.strip()

# Example
print("Before: ", data[0]['response'])
print("After: ", preprocess_sentence(data[0]['response']))


Before:  @USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..
After:  i don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..


In [28]:
# data loader
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

class SarcasmDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        #원하는 Hyperparameter를 추가하여도 된다.
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        response = preprocess_sentence(self.data[index]['response'])
        label = 1 if self.data[index]['label'] == 'SARCASM' else 0
        context = preprocess_sentence(self.data[index]['context'])

        #적절한 Context와 Response를 결합하여 Input으로 사용하도록 한다. Context 길이 또한 고려하여야하는 Hyperparameter이다.
        input = '[CLS]' + context + '[SEP]' + response + '[SEP]'

        # Tokenizing
        input_ids = self.tokenizer.encode(input, add_special_tokens=False, max_length=self.max_len, truncation=True)
        token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
        attention_mask = [1] * len(input_ids)

        return input_ids, token_type_ids, attention_mask, label

# Example
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset = SarcasmDataset(data, tokenizer, 256)

print("original_sentence", data[0]['context'][-1], data[0]['response'])
print("Input_ids: ", dataset[0][0])
print("Token_type_ids: ", dataset[0][1])
print("Attention_mask: ", dataset[0][2])
print("Label: ", dataset[0][3])

original_sentence @USER If your child isn't named Barron ... #BeBest Melania couldn't care less . Fact . 💯 @USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..
Input_ids:  [101, 2065, 2115, 2775, 3475, 1005, 1056, 2315, 23594, 1012, 1012, 1012, 1001, 2022, 12681, 2102, 11463, 13241, 2481, 1005, 1056, 2729, 2625, 1012, 2755, 1012, 100, 102, 1045, 2123, 1005, 1056, 2131, 2023, 1012, 1012, 5525, 2017, 2079, 2729, 2030, 2017, 2052, 1005, 2310, 2333, 2157, 2247, 1012, 1012, 2612, 2017, 2787, 2000, 2729, 1998, 18792, 2014, 1012, 1012, 102]
Token_type_ids:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Attention_mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [29]:
# collate function considering padding
def collate_fn(batch):
    max_len = max(max(len(item[0]), len(item[1]), len(item[2])) for item in batch)  # Find max length in batch

    input_ids, token_type_ids, attention_mask, labels = [], [], [], []
    for item in batch:
        # Padding each sequence to the max length
        input_ids.append(item[0] + [0] * (max_len - len(item[0])))
        token_type_ids.append(item[1] + [0] * (max_len - len(item[1])))
        attention_mask.append(item[2] + [0] * (max_len - len(item[2])))
        labels.append(item[3])

    # Convert lists to tensors
    input_ids = torch.LongTensor(input_ids)
    token_type_ids = torch.LongTensor(token_type_ids)
    attention_mask = torch.LongTensor(attention_mask)
    labels = torch.LongTensor(labels)

    return input_ids, token_type_ids, attention_mask, labels

# Example
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
for x in dataloader:
    print("Input_ids: ", x[0])
    print("Token_type_ids: ", x[1])
    print("Attention_mask: ", x[2])
    print("Label: ", x[3])
    break

Input_ids:  tensor([[  101,  1012,  1999,  3365, 13867,  1010,  7896,  1004,  7636,  1010,
          1045,  2562,  6911,  2075,  2008,  1996,  4471,  1523,  1045,  2097,
          4847,  2115,  3606,  1010,  3531,  4847,  2026,  3606,  1524,  2003,
          1996,  2069,  3978,  2005,  3412, 24873,  9048, 16173,  3401,  1004,
          2003,  1996,  2168,  6958,  2008, 15770,  2015,  2256,  7578,  7072,
          1012,   102,  3519,  7164,  1996, 15766,  1997, 15029,  9841, 27292,
         16585,  1004, 12725,  7560,  1011,  5152,  8499,  2004,  1996,  6827,
          3653,  1011,  2128, 24871,  2005, 10615,  1001, 24264, 12458,  5063,
          8303,  1004,  1026, 24471,  2140,  1028,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  1045,  4299,  2017,  1996,  2168,  2026,  4086,  2767,   100,
  

In [30]:
# Train data
train_data = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Test data
with open("./sarcasm/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl", 'r', encoding="utf-8") as file:
    datalines = file.readlines()

test_data = []
for line in datalines:
    test_data.append(json.loads(line))

test_dataset = SarcasmDataset(test_data, tokenizer, 256)
test_data = DataLoader(test_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

## 3. Modeling

In [31]:
from transformers import AutoModel, AutoConfig
import torch.nn as nn

class SarcasmDetectionModel(nn.Module):
    def __init__(self, model_name, num_labels):
        super(SarcasmDetectionModel, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
        self.encoder = AutoModel.from_pretrained(model_name, config=self.config)

        self.classifier = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        outputs = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        logits = self.classifier(pooled_output)

        return logits

# Example
model = SarcasmDetectionModel("bert-base-uncased", num_labels=2).cuda()

for x in train_data:
    input_ids, token_type_ids, attention_mask, labels = x
    logits = model(input_ids.cuda(), token_type_ids=token_type_ids.cuda(), attention_mask=attention_mask.cuda())
    print("Input_ids: ", input_ids)
    print("Token_type_ids: ", token_type_ids)
    print("Attention_mask: ", attention_mask)
    print("Logits: ", logits)
    print("Result: ", torch.argmax(logits, dim=-1))
    print("Label: ", labels)
    break

Input_ids:  tensor([[  101,  8112, 18465,  2094,  1037,  2149,  6926,  1012,  2302,  7740,
          6226,  1012,  1045,  1521,  1049,  2469,  2017,  2020,  2039,  1999,
          2608,  2055,  2008,  1012,   100,   100,  1012,  2821,  2232,  1012,
          1998,  1037,  5030,  1012,  4533,  2015,   100,  2004,  2092,  1012,
          8224,  2009,  1012,   102,  2065,  2009,  1521,  1055,  2006,  1996,
          4274,  1998, 13366, 14074,  2015,  2343,  8112,  1010,  2009,  2442,
          2022,  2995,  1012,  1012,  1012,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2127,  2017,  2131,  5236,  3980,  2055,  2009,  4826,  8840,
          2140,   102,  2613,  3160,  1024,  2079,  2017,  5382,  2008,  2023,
          2003,  1037, 12354,  4070, 19545, 17688,  2099,  1998,  1996,  8398,
          4748, 10020,  1029,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [32]:
# Optimizer, In here you can use any optimizer you want.
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-6)

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Example
input_ids, token_type_ids, attention_mask, labels = next(iter(train_data))
output = model(input_ids.cuda(), token_type_ids.cuda(), attention_mask.cuda())
print("Output: ", output)
print("Gold: ", labels)

Output:  tensor([[-0.1785,  0.2645],
        [-0.1580,  0.1745],
        [-0.1490,  0.3178],
        [-0.3425,  0.3970],
        [-0.3420,  0.3694],
        [-0.2812,  0.4242],
        [-0.3403,  0.3712],
        [-0.3075,  0.5280]], device='cuda:0', grad_fn=<AddmmBackward0>)
Gold:  tensor([0, 1, 1, 1, 1, 0, 0, 0])


## 4. Training

In [33]:
# Training
from tqdm import tqdm

epochs = 20
for epoch in range(epochs):
    model.train()
    for input_ids, token_type_ids, attention_mask, labels in tqdm(train_data):
        optimizer.zero_grad()

        output = model(input_ids.cuda(), token_type_ids.cuda(), attention_mask.cuda())
        loss = loss_fn(output, labels.cuda())
        loss.backward()
        optimizer.step()

    model.eval()
    total = 0
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for input_ids, token_type_ids, attention_mask, labels in tqdm(test_data):
        output = model(input_ids.cuda(), token_type_ids.cuda(), attention_mask.cuda())
        prediction = torch.argmax(output, dim=-1)
        tp += ((prediction == 1) & (labels.cuda() == 1)).sum().item()
        fp += ((prediction == 1) & (labels.cuda() == 0)).sum().item()
        fn += ((prediction == 0) & (labels.cuda() == 1)).sum().item()
        tn += ((prediction == 0) & (labels.cuda() == 0)).sum().item()
        total += len(labels)

    print("Accuracy: ", (tp + tn) / total)
    print("Precision: ", tp / (tp + fp))
    print("Recall: ", tp / (tp + fn))
    print("F1: ", 2 * tp / (2 * tp + fp + fn))

100%|██████████| 225/225 [00:05<00:00, 39.22it/s]


Accuracy:  0.6455555555555555
Precision:  0.6077302631578947
Recall:  0.8211111111111111
F1:  0.6984877126654064


100%|██████████| 225/225 [00:05<00:00, 39.64it/s]


Accuracy:  0.6472222222222223
Precision:  0.6328986960882648
Recall:  0.7011111111111111
F1:  0.665260938323669


100%|██████████| 225/225 [00:05<00:00, 40.18it/s]


...

Accuracy:  0.7033333333333334
Precision:  0.7015418502202643
Recall:  0.7077777777777777
F1:  0.7046460176991151


100%|██████████| 225/225 [00:05<00:00, 39.56it/s]


Accuracy:  0.6972222222222222
Precision:  0.6882290562036055
Recall:  0.7211111111111111
F1:  0.7042864894194248


 49%|████▉     | 305/625 [00:25<00:26, 12.01it/s]


KeyboardInterrupt: 